# FOOD-101-MINI Classifier

### Autor: Filip Gębala

#### Import bibliotek

In [1]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

#### Wczytywanie i przygotowanie danych

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset = datasets.ImageFolder('Data/food-101-tiny/train', transform=train_transform)
test_dataset = datasets.ImageFolder('Data/food-101-tiny/valid', transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

#### Architektura modelu

In [3]:
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, num_classes):
        super(Model, self).__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), 
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.block_2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1), 
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.block_3 = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1), 
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.global_avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.block_3(x)
        x = self.global_avg_pool(x)  # teraz 256 x 1 x 1
        logits = self.classifier(x)
        return logits


#### Trening modelu

In [ ]:
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
model = Model(num_classes=10).to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.001)
criterion = nn.CrossEntropyLoss()

# === EARLY STOPPING PARAMETRY ===
best_val_loss = float('inf')
epochs_without_improvement = 0
patience = 7  # liczba epok bez poprawy

for epoch in range(50):
    model.train()
    running_loss = 0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

    train_acc = correct_train / total_train

    # === EWALUACJA ===
    model.eval()
    correct = 0
    total = 0
    val_loss = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    avg_train_loss = running_loss / len(train_loader)
    avg_val_loss = val_loss / len(test_loader)

    print(f"Epoch {epoch+1}: Train Acc: {train_acc*100:.2f}% | Val Acc: {val_acc*100:.2f}% | "
          f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    # === EARLY STOPPING SPRAWDZENIE ===
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_without_improvement = 0
        # (opcjonalnie: torch.save(model.state_dict(), 'best_model.pth'))
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"Early stopping at epoch {epoch+1} — validation loss didn't improve for {patience} epochs.")
            break

Using device: cuda
Epoch 1: Train Acc: 10.60% | Val Acc: 16.60% | Train Loss: 2.2883 | Val Loss: 2.2550
Epoch 2: Train Acc: 19.93% | Val Acc: 28.20% | Train Loss: 2.2246 | Val Loss: 2.1800
Epoch 3: Train Acc: 22.60% | Val Acc: 29.60% | Train Loss: 2.1753 | Val Loss: 2.1284
Epoch 4: Train Acc: 25.40% | Val Acc: 29.20% | Train Loss: 2.1269 | Val Loss: 2.0965
Epoch 5: Train Acc: 26.20% | Val Acc: 31.40% | Train Loss: 2.0992 | Val Loss: 2.0633
Epoch 6: Train Acc: 27.07% | Val Acc: 32.00% | Train Loss: 2.0910 | Val Loss: 2.0534
Epoch 7: Train Acc: 28.47% | Val Acc: 34.40% | Train Loss: 2.0596 | Val Loss: 2.0323


In [5]:
from sklearn.metrics import accuracy_score, classification_report
import torch

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = outputs.argmax(1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

print(f'Accuracy: {accuracy_score(all_labels, all_preds)*100:.2f}%')
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

Accuracy: 45.20%
              precision    recall  f1-score   support

   apple_pie       0.35      0.40      0.37        50
    bibimbap       0.44      0.72      0.55        50
     cannoli       0.42      0.32      0.36        50
     edamame       0.92      0.94      0.93        50
     falafel       0.39      0.30      0.34        50
french_toast       0.65      0.44      0.52        50
   ice_cream       0.47      0.38      0.42        50
       ramen       0.37      0.56      0.45        50
       sushi       0.25      0.36      0.30        50
    tiramisu       0.38      0.10      0.16        50

    accuracy                           0.45       500
   macro avg       0.47      0.45      0.44       500
weighted avg       0.47      0.45      0.44       500

